# Problem Statement 

#### To determine the best model/feature for predicting Sale Price for houses in Ames area.



# Background 

#### We are given a dataset of housing prices and possibily, related variables that we are to use to predict housing prices in Ames, Iowa. 

## Import Necessary Python Libraries 

In [135]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.metrics import r2_score
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler
import pandas_profiling

### Import Train Data 

In [136]:
# Import train data
df = pd.read_csv("../datasets/train.csv", index_col= 'Id')

In [137]:
# View of data structure 
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2051 entries, 109 to 10
Data columns (total 80 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   PID              2051 non-null   int64  
 1   MS SubClass      2051 non-null   int64  
 2   MS Zoning        2051 non-null   object 
 3   Lot Frontage     1721 non-null   float64
 4   Lot Area         2051 non-null   int64  
 5   Street           2051 non-null   object 
 6   Alley            140 non-null    object 
 7   Lot Shape        2051 non-null   object 
 8   Land Contour     2051 non-null   object 
 9   Utilities        2051 non-null   object 
 10  Lot Config       2051 non-null   object 
 11  Land Slope       2051 non-null   object 
 12  Neighborhood     2051 non-null   object 
 13  Condition 1      2051 non-null   object 
 14  Condition 2      2051 non-null   object 
 15  Bldg Type        2051 non-null   object 
 16  House Style      2051 non-null   object 
 17  Overall Qual  

In [138]:
df.head()

,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
109,533352170,60,RL,NaN,13517,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,3,2010,WD,130500
544,531379050,60,RL,43.0,11492,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,4,2009,WD,220000
153,535304180,20,RL,68.0,7922,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,1,2010,WD,109000
318,916386060,60,RL,73.0,9802,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,4,2010,WD,174000
255,906425045,50,RL,82.0,14235,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,3,2010,WD,138500


In [139]:
df.shape

(2051, 80)

### Drop data that are significantly incomplete by >60%


In [140]:
df = df.drop('Alley', axis = 1)

In [141]:
df = df.drop('Pool QC', axis = 1)

In [142]:
df = df.drop("Fence", axis = 1)

In [143]:
df = df.drop("Misc Feature", axis = 1)

In [144]:
df = df.drop("Fireplace Qu", axis = 1)

In [145]:
df = df.drop("PID", axis = 1)

In [146]:
df.head()

,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,...,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
109,60,RL,NaN,13517,Pave,IR1,Lvl,AllPub,CulDSac,Gtl,...,44,0,0,0,0,0,3,2010,WD,130500
544,60,RL,43.0,11492,Pave,IR1,Lvl,AllPub,CulDSac,Gtl,...,74,0,0,0,0,0,4,2009,WD,220000
153,20,RL,68.0,7922,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,52,0,0,0,0,0,1,2010,WD,109000
318,60,RL,73.0,9802,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,0,4,2010,WD,174000
255,50,RL,82.0,14235,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,59,0,0,0,0,0,3,2010,WD,138500


In [147]:
sns.displot(df['SalePrice']);
print('Skewness: %f' %  df['SalePrice'].skew())
print('Kurtosis: %f' %  df['SalePrice'].kurtosis())

# The distribution of the sale price is skewed to the right.

Skewness: 1.557551
Kurtosis: 3.579003


### Check what are the data and its columns available from the dataset.

In [148]:
for col in (list(df.columns)):
    print(col, ":", df[col].unique())

MS SubClass : [ 60  20  50 180 160  70 120 190  85  30  90  80  75  45  40 150]
MS Zoning : ['RL' 'RM' 'FV' 'C (all)' 'A (agr)' 'RH' 'I (all)']
Lot Frontage : [ nan  43.  68.  73.  82. 137.  35.  70.  21.  64. 120.  24.  74.  93.
  34.  80.  71.  72. 109.  40. 103. 100.  92.  65.  75.  60.  30.  79.
  41. 105. 107.  81.  36.  63.  32.  94.  44.  50.  48.  67.  88.  83.
  53.  58.  57.  52.  87. 134.  56.  54. 140.  78.  85.  90.  96.  62.
  49.  59. 155.  91.  61.  86. 128.  77.  42.  89.  51.  69.  55. 112.
  76. 125.  98. 113. 102.  22. 122.  84. 119. 118.  66.  95. 108. 195.
 106.  39. 110. 130.  97.  45.  37. 123.  38. 129. 115.  47. 114. 104.
  46. 121. 124. 313. 141. 101.  99. 160. 174.  26. 144. 138. 111.  25.
  33. 200. 150. 117. 153. 116. 135.]
Lot Area : [13517 11492  7922 ... 12444 11449  7558]
Street : ['Pave' 'Grvl']
Lot Shape : ['IR1' 'Reg' 'IR2' 'IR3']
Land Contour : ['Lvl' 'HLS' 'Bnk' 'Low']
Utilities : ['AllPub' 'NoSeWa' 'NoSewr']
Lot Config : ['CulDSac' 'Inside' 'Corn

### Renaming all capitalised titles in columns and removing spaces 


In [149]:
n_col = df.columns
rename_dict = {}
for col in n_col: 
    rename_dict[col] = col.lower().replace(' ', "")

df.rename(columns = rename_dict, inplace = True)
df.columns

Index(['mssubclass', 'mszoning', 'lotfrontage', 'lotarea', 'street',
       'lotshape', 'landcontour', 'utilities', 'lotconfig', 'landslope',
       'neighborhood', 'condition1', 'condition2', 'bldgtype', 'housestyle',
       'overallqual', 'overallcond', 'yearbuilt', 'yearremod/add', 'roofstyle',
       'roofmatl', 'exterior1st', 'exterior2nd', 'masvnrtype', 'masvnrarea',
       'exterqual', 'extercond', 'foundation', 'bsmtqual', 'bsmtcond',
       'bsmtexposure', 'bsmtfintype1', 'bsmtfinsf1', 'bsmtfintype2',
       'bsmtfinsf2', 'bsmtunfsf', 'totalbsmtsf', 'heating', 'heatingqc',
       'centralair', 'electrical', '1stflrsf', '2ndflrsf', 'lowqualfinsf',
       'grlivarea', 'bsmtfullbath', 'bsmthalfbath', 'fullbath', 'halfbath',
       'bedroomabvgr', 'kitchenabvgr', 'kitchenqual', 'totrmsabvgrd',
       'functional', 'fireplaces', 'garagetype', 'garageyrblt', 'garagefinish',
       'garagecars', 'garagearea', 'garagequal', 'garagecond', 'paveddrive',
       'wooddecksf', 'openporchsf

In [150]:
df['saleprice'].astype(str).astype(int)

Id
109     130500
544     220000
153     109000
318     174000
255     138500
         ...  
1587    298751
785      82500
916     177000
639     144000
10      189000
Name: saleprice, Length: 2051, dtype: int32

### Identifying categorical and continuous variables

In [151]:
# Populating a list of categorical variables

cat_col = df.columns
cat_list = list(df.select_dtypes(include = 'object').columns)

In [152]:
# Iterating through the categorical variable list to print out the variables and the unique entries

for col in cat_list:
    print(col, df[col].unique())
    
# Creating a dictionary of categorical variables and the number of categories 

cat_options = {}
for col in cat_list:
    cat_options[col] = len(df[col].unique())

mszoning ['RL' 'RM' 'FV' 'C (all)' 'A (agr)' 'RH' 'I (all)']
street ['Pave' 'Grvl']
lotshape ['IR1' 'Reg' 'IR2' 'IR3']
landcontour ['Lvl' 'HLS' 'Bnk' 'Low']
utilities ['AllPub' 'NoSeWa' 'NoSewr']
lotconfig ['CulDSac' 'Inside' 'Corner' 'FR2' 'FR3']
landslope ['Gtl' 'Sev' 'Mod']
neighborhood ['Sawyer' 'SawyerW' 'NAmes' 'Timber' 'Edwards' 'OldTown' 'BrDale'
 'CollgCr' 'Somerst' 'Mitchel' 'StoneBr' 'NridgHt' 'Gilbert' 'Crawfor'
 'IDOTRR' 'NWAmes' 'Veenker' 'MeadowV' 'SWISU' 'NoRidge' 'ClearCr'
 'Blmngtn' 'BrkSide' 'NPkVill' 'Blueste' 'GrnHill' 'Greens' 'Landmrk']
condition1 ['RRAe' 'Norm' 'PosA' 'Artery' 'Feedr' 'PosN' 'RRAn' 'RRNe' 'RRNn']
condition2 ['Norm' 'RRNn' 'Feedr' 'Artery' 'PosA' 'PosN' 'RRAe' 'RRAn']
bldgtype ['1Fam' 'TwnhsE' 'Twnhs' '2fmCon' 'Duplex']
housestyle ['2Story' '1Story' '1.5Fin' 'SFoyer' 'SLvl' '2.5Unf' '2.5Fin' '1.5Unf']
roofstyle ['Gable' 'Hip' 'Flat' 'Mansard' 'Shed' 'Gambrel']
roofmatl ['CompShg' 'WdShngl' 'Tar&Grv' 'WdShake' 'Membran' 'ClyTile']
exterior1st ['Hd

In [153]:
# List continuous variables

cont_col = df.columns
cont_list = list(df.select_dtypes(include = 'int64').columns)

In [154]:
df[cont_list].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2051 entries, 109 to 10
Data columns (total 26 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   mssubclass     2051 non-null   int64
 1   lotarea        2051 non-null   int64
 2   overallqual    2051 non-null   int64
 3   overallcond    2051 non-null   int64
 4   yearbuilt      2051 non-null   int64
 5   yearremod/add  2051 non-null   int64
 6   1stflrsf       2051 non-null   int64
 7   2ndflrsf       2051 non-null   int64
 8   lowqualfinsf   2051 non-null   int64
 9   grlivarea      2051 non-null   int64
 10  fullbath       2051 non-null   int64
 11  halfbath       2051 non-null   int64
 12  bedroomabvgr   2051 non-null   int64
 13  kitchenabvgr   2051 non-null   int64
 14  totrmsabvgrd   2051 non-null   int64
 15  fireplaces     2051 non-null   int64
 16  wooddecksf     2051 non-null   int64
 17  openporchsf    2051 non-null   int64
 18  enclosedporch  2051 non-null   int64
 19  3ssnpo

In [155]:
df[cont_list].describe()

,mssubclass,lotarea,overallqual,overallcond,yearbuilt,yearremod/add,1stflrsf,2ndflrsf,lowqualfinsf,grlivarea,...,wooddecksf,openporchsf,enclosedporch,3ssnporch,screenporch,poolarea,miscval,mosold,yrsold,saleprice
count,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,...,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000
mean,57.008776,10065.208191,6.112140,5.562165,1971.708922,1984.190151,1164.488055,329.329108,5.512921,1499.330083,...,93.833740,47.556802,22.571916,2.591419,16.511458,2.397855,51.574354,6.219893,2007.775719,181469.701609
std,42.824223,6742.488909,1.426271,1.104497,30.177889,21.036250,396.446923,425.671046,51.068870,500.447829,...,128.549416,66.747241,59.845110,25.229615,57.374204,37.782570,573.393985,2.744736,1.312014,79258.659352
min,20.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,334.000000,0.000000,0.000000,334.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,12789.000000
25%,20.000000,7500.000000,5.000000,5.000000,1953.500000,1964.500000,879.500000,0.000000,0.000000,1129.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,2007.000000,129825.000000
50%,50.000000,9430.000000,6.000000,5.000000,1974.000000,1993.000000,1093.000000,0.000000,0.000000,1444.000000,...,0.000000,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,162500.000000
75%,70.000000,11513.500000,7.000000,6.000000,2001.000000,2004.000000,1405.000000,692.500000,0.000000,1728.500000,...,168.000000,70.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,190.000000,159000.000000,10.000000,9.000000,2010.000000,2010.000000,5095.000000,1862.000000,1064.000000,5642.000000,...,1424.000000,547.000000,432.000000,508.000000,490.000000,800.000000,17000.000000,12.000000,2010.000000,611657.000000


### Exploring correlations between continuous variables

In [156]:
plt.figure(figsize = (5,20))

ax= sns.heatmap(df.corr()[cont_list][['saleprice']].sort_values(by = 'saleprice',ascending = False),annot=True,  annot_kws={"size": 20}, cmap='PuBuGn')
ax.set_ylim(37,0)
plt.yticks(size = 14)
plt.xticks(size = 14)
plt.title('Heatmap of Continuous Variable Correlation to Sale Price', size = 20)

Text(0.5, 1.0, 'Heatmap of Continuous Variable Correlation to Sale Price')

In [157]:
# P value of the correlations above 

def calculate_pvalues(df):
    df = df.dropna()._get_numeric_data()
    dfcols = pd.DataFrame(columns=df.columns)
    pvalues = dfcols.transpose().join(dfcols, how='outer')
    for r in df.columns:
        for c in df.columns:
            pvalues[r][c] = round(stats.pearsonr(df[r], df[c])[1], 4)
    return pvalues

calculate_pvalues(df[cont_list])['saleprice'].sort_values(ascending = False)

miscval          0.7385
yrsold           0.4914
poolarea         0.2956
mosold           0.1383
lowqualfinsf     0.0596
3ssnporch        0.0273
mssubclass       0.0001
totrmsabvgrd        0.0
screenporch         0.0
enclosedporch       0.0
openporchsf         0.0
wooddecksf          0.0
fireplaces          0.0
kitchenabvgr        0.0
lotarea             0.0
bedroomabvgr        0.0
halfbath            0.0
fullbath            0.0
grlivarea           0.0
2ndflrsf            0.0
1stflrsf            0.0
yearremod/add       0.0
yearbuilt           0.0
overallcond         0.0
overallqual         0.0
saleprice           0.0
Name: saleprice, dtype: object

In [158]:
# Function to find collinearity between expected collinear variables

def viffinder(varlist):
    vif = pd.DataFrame()
    vif["variables"] = varlist
    vif["VIF"] = \
    [variance_inflation_factor(df[varlist].dropna().values,\
                               i) for i in range(df[varlist].shape[1])]
    print(vif)

In [159]:
# Top few variables correlation to saleprice, positive correlations were chosen over negative as correlations had smaller correlation coefficients.

top_corr_list = list(df.corr()[cont_list][['saleprice']].sort_values(by = 'saleprice',\
                                                ascending = False).nlargest(30,['saleprice']).index)
print(top_corr_list)

['saleprice', 'overallqual', 'grlivarea', 'garagearea', 'garagecars', 'totalbsmtsf', '1stflrsf', 'yearbuilt', 'yearremod/add', 'fullbath', 'garageyrblt', 'masvnrarea', 'totrmsabvgrd', 'fireplaces', 'bsmtfinsf1', 'lotfrontage', 'openporchsf', 'wooddecksf', 'lotarea', 'bsmtfullbath', 'halfbath', '2ndflrsf', 'bsmtunfsf', 'bedroomabvgr', 'screenporch', '3ssnporch', 'mosold', 'poolarea', 'bsmtfinsf2', 'miscval']


In [160]:
mask = np.zeros_like(df[top_corr_list].corr())
mask[np.triu_indices_from(mask)] = True

plt.figure(figsize = (40,40))
plt.title('Heatmap of top 30 correlated values', size = 20)
ax = sns.heatmap(df[top_corr_list].corr(), annot = True, annot_kws={"size": 20}, mask = mask)
ax.set_ylim(30,0)
plt.yticks(size = 14, rotation = 90)
plt.xticks(size = 14)

(array([ 0.5,  1.5,  2.5,  3.5,  4.5,  5.5,  6.5,  7.5,  8.5,  9.5, 10.5,
        11.5, 12.5, 13.5, 14.5, 15.5, 16.5, 17.5, 18.5, 19.5, 20.5, 21.5,
        22.5, 23.5, 24.5, 25.5, 26.5, 27.5, 28.5, 29.5]),
 [Text(0.5, 0, 'saleprice'),
  Text(1.5, 0, 'overallqual'),
  Text(2.5, 0, 'grlivarea'),
  Text(3.5, 0, 'garagearea'),
  Text(4.5, 0, 'garagecars'),
  Text(5.5, 0, 'totalbsmtsf'),
  Text(6.5, 0, '1stflrsf'),
  Text(7.5, 0, 'yearbuilt'),
  Text(8.5, 0, 'yearremod/add'),
  Text(9.5, 0, 'fullbath'),
  Text(10.5, 0, 'garageyrblt'),
  Text(11.5, 0, 'masvnrarea'),
  Text(12.5, 0, 'totrmsabvgrd'),
  Text(13.5, 0, 'fireplaces'),
  Text(14.5, 0, 'bsmtfinsf1'),
  Text(15.5, 0, 'lotfrontage'),
  Text(16.5, 0, 'openporchsf'),
  Text(17.5, 0, 'wooddecksf'),
  Text(18.5, 0, 'lotarea'),
  Text(19.5, 0, 'bsmtfullbath'),
  Text(20.5, 0, 'halfbath'),
  Text(21.5, 0, '2ndflrsf'),
  Text(22.5, 0, 'bsmtunfsf'),
  Text(23.5, 0, 'bedroomabvgr'),
  Text(24.5, 0, 'screenporch'),
  Text(25.5, 0, '3ssnporch'),

In [161]:
# Garage related variables were chosen from the top correlated list and plotted

garage_trio = [cat for cat in top_corr_list if cat.find('garage') >= 0 ]

plt.figure(figsize = (5,5))
ax = sns.heatmap(df[garage_trio].corr(), annot = True, annot_kws={"size": 20})
ax.set_ylim(3,0)
plt.yticks(size = 14)
plt.xticks(size = 14)

(array([0.5, 1.5, 2.5]),
 [Text(0.5, 0, 'garagearea'),
  Text(1.5, 0, 'garagecars'),
  Text(2.5, 0, 'garageyrblt')])

In [162]:
# Finding the VIF

viffinder(garage_trio)

     variables        VIF
0   garagearea  28.954657
1   garagecars  33.658878
2  garageyrblt   9.930975


In [163]:
# Variables relating to year of last build were tested 

year_var = [cat for cat in top_corr_list if cat.find('year') >= 0 ]

plt.figure(figsize = (7,7))
ax = sns.heatmap(df[year_var].corr(), annot = True, annot_kws={"size": 20})
ax.set_ylim(2,0)
plt.yticks(size = 14)
plt.xticks(size = 14)

(array([0.5, 1.5]), [Text(0.5, 0, 'yearbuilt'), Text(1.5, 0, 'yearremod/add')])

In [164]:
# Finding the VIF

viffinder(year_var)

       variables          VIF
0      yearbuilt  7023.224832
1  yearremod/add  7023.224832


In [166]:
# Basement related variables were listed together

basementvar =[cat for cat in top_corr_list if cat.find('bsmt') >= 0 ]
basementvar

['totalbsmtsf', 'bsmtfinsf1', 'bsmtfullbath', 'bsmtunfsf', 'bsmtfinsf2']

In [167]:
# Basement related variables were plotted for correlation.

plt.figure(figsize = (10,10))
ax = sns.heatmap(df[basementvar].corr(), annot = True, annot_kws={"size": 20})
ax.set_ylim(5,0)
plt.yticks(size = 14)
plt.xticks(size = 14)

(array([0.5, 1.5, 2.5, 3.5, 4.5]),
 [Text(0.5, 0, 'totalbsmtsf'),
  Text(1.5, 0, 'bsmtfinsf1'),
  Text(2.5, 0, 'bsmtfullbath'),
  Text(3.5, 0, 'bsmtunfsf'),
  Text(4.5, 0, 'bsmtfinsf2')])

In [168]:
viffinder(basementvar)

      variables       VIF
0   totalbsmtsf       inf
1    bsmtfinsf1       inf
2  bsmtfullbath  3.012996
3     bsmtunfsf       inf
4    bsmtfinsf2       inf


C:\Users\singh\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Users\singh\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Users\singh\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Users\singh\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


In [169]:
leftover = set(top_corr_list) - set(basementvar) - set(garage_trio) - set(year_var)
leftover = list(leftover)
leftover

['fireplaces',
 '1stflrsf',
 'mosold',
 'openporchsf',
 'totrmsabvgrd',
 'overallqual',
 'bedroomabvgr',
 '3ssnporch',
 'masvnrarea',
 'screenporch',
 'poolarea',
 'saleprice',
 'grlivarea',
 'lotarea',
 'halfbath',
 'lotfrontage',
 'fullbath',
 '2ndflrsf',
 'miscval',
 'wooddecksf']

In [170]:
# Mask up 
mask = np.zeros_like(df[leftover].corr())
mask[np.triu_indices_from(mask)] = True

## the remaining variables were plotted for correlation.
plt.figure(figsize = (25,25))
ax = sns.heatmap(df[leftover].corr(), annot = True, annot_kws={"size": 18}, mask = mask)
ax.set_ylim(20,0)
plt.yticks(size = 17)
plt.xticks(size = 17)
ax.set_yticklabels(leftover,rotation = 45)

[Text(0, 0.5, 'fireplaces'),
 Text(0, 1.5, '1stflrsf'),
 Text(0, 2.5, 'mosold'),
 Text(0, 3.5, 'openporchsf'),
 Text(0, 4.5, 'totrmsabvgrd'),
 Text(0, 5.5, 'overallqual'),
 Text(0, 6.5, 'bedroomabvgr'),
 Text(0, 7.5, '3ssnporch'),
 Text(0, 8.5, 'masvnrarea'),
 Text(0, 9.5, 'screenporch'),
 Text(0, 10.5, 'poolarea'),
 Text(0, 11.5, 'saleprice'),
 Text(0, 12.5, 'grlivarea'),
 Text(0, 13.5, 'lotarea'),
 Text(0, 14.5, 'halfbath'),
 Text(0, 15.5, 'lotfrontage'),
 Text(0, 16.5, 'fullbath'),
 Text(0, 17.5, '2ndflrsf'),
 Text(0, 18.5, 'miscval'),
 Text(0, 19.5, 'wooddecksf')]

In [171]:
viffinder(leftover)

       variables         VIF
0     fireplaces    2.576286
1       1stflrsf  586.494631
2         mosold    5.721490
3    openporchsf    1.812263
4   totrmsabvgrd   65.440977
5    overallqual   36.446228
6   bedroomabvgr   25.558351
7      3ssnporch    1.021344
8     masvnrarea    1.896760
9    screenporch    1.170186
10      poolarea    1.102490
11     saleprice   26.200634
12     grlivarea  921.262549
13       lotarea    8.558904
14      halfbath    2.657137
15   lotfrontage   16.113018
16      fullbath   18.095659
17      2ndflrsf  108.431734
18       miscval    1.131738
19    wooddecksf    1.871388


In [172]:
# Final correlation picked list of variables

final_corr_list = ['overallqual', 'masvnrarea', 'poolarea', 'halfbath', \
                   'fireplaces', 'openporchsf', 'grlivarea', 'screenporch', \
                   'lotarea', '3ssnporch', 'bsmtfullbath', 'totalbsmtsf', \
                  'garagearea', 'yearbuilt']

In [173]:
viffinder(final_corr_list)

       variables        VIF
0    overallqual  43.928379
1     masvnrarea   1.811127
2       poolarea   1.040005
3       halfbath   2.210379
4     fireplaces   2.605008
5    openporchsf   1.763804
6      grlivarea  22.747272
7    screenporch   1.124378
8        lotarea   4.032884
9      3ssnporch   1.017818
10  bsmtfullbath   1.928679
11   totalbsmtsf  13.496182
12    garagearea   9.862637
13     yearbuilt  27.950242


In [174]:
df[final_corr_list].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2051 entries, 109 to 10
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   overallqual   2051 non-null   int64  
 1   masvnrarea    2029 non-null   float64
 2   poolarea      2051 non-null   int64  
 3   halfbath      2051 non-null   int64  
 4   fireplaces    2051 non-null   int64  
 5   openporchsf   2051 non-null   int64  
 6   grlivarea     2051 non-null   int64  
 7   screenporch   2051 non-null   int64  
 8   lotarea       2051 non-null   int64  
 9   3ssnporch     2051 non-null   int64  
 10  bsmtfullbath  2049 non-null   float64
 11  totalbsmtsf   2050 non-null   float64
 12  garagearea    2050 non-null   float64
 13  yearbuilt     2051 non-null   int64  
dtypes: float64(4), int64(10)
memory usage: 240.4 KB


### Statistical inference tests to determine a possible relationship between categorical variables and price


In [175]:
# selecting categories for t-test 

ttest_list = []
for catkey, catvalues in cat_options.items(): 
    if catvalues == 2:
        ttest_list.append(catkey)
ttest_list

['street', 'centralair']

In [176]:
# Performing t-test on columns in ttest_list

cat_result_dict = {}
for tcand in ttest_list:
    cat1 = df[df[tcand]==(df[tcand].unique()[0])]['saleprice']
    cat2 = df[df[tcand]==(df[tcand].unique()[1])]['saleprice']
    t, p = stats.ttest_ind(cat1, cat2)
    sig = ''
    if p <= 0.05:
        sig = 'Y'
    else: 
        sig = 'N'
    cat_result_dict[tcand] = {df[tcand].unique()[0]:cat1.mean(),\
                              df[tcand].unique()[1]:cat2.mean(), \
                              'statistic': t, 'P': p, 'significance': sig}

cat_result_dict

{'street': {'Pave': 181793.56555772995,
  'Grvl': 86901.42857142857,
  'statistic': 3.169173295799292,
  'P': 0.0015512882730560206,
  'significance': 'Y'},
 'centralair': {'Y': 187441.50366492147,
  'N': 100575.0780141844,
  'statistic': 13.068548592323888,
  'P': 1.5113094019787156e-37,
  'significance': 'Y'}}

### For the above, both street and centralair variables showed a significant difference in mean salesprice between categories.

#### Houses with paved streets had higher sale price as compared to gravel.
#### Houses with central airconditioning had higher sale price as compared to those without

In [177]:
# Removing t-testlist variables from cat_list so that it will not be run again

def rem_entry_list(list1, list2):
    for var1 in list1: 
        if var1 in list2:
            list2.remove(var1)
            next
        return list2
rem_entry_list(ttest_list, cat_list)

['mszoning',
 'lotshape',
 'landcontour',
 'utilities',
 'lotconfig',
 'landslope',
 'neighborhood',
 'condition1',
 'condition2',
 'bldgtype',
 'housestyle',
 'roofstyle',
 'roofmatl',
 'exterior1st',
 'exterior2nd',
 'masvnrtype',
 'exterqual',
 'extercond',
 'foundation',
 'bsmtqual',
 'bsmtcond',
 'bsmtexposure',
 'bsmtfintype1',
 'bsmtfintype2',
 'heating',
 'heatingqc',
 'centralair',
 'electrical',
 'kitchenqual',
 'functional',
 'garagetype',
 'garagefinish',
 'garagequal',
 'garagecond',
 'paveddrive',
 'saletype']

In [178]:
# Defining a function to perform an analysis of variance(ANOVA) through the list 

def ANOVA_list(searchlist, dataframe, resultdict):
    a = {}
    for var in searchlist:
        var_unique_list = []
        count = 0
        resultdict[var] = {}
        for i in range(len(dataframe[var].unique())):
            count += 1 
            if pd.isnull(dataframe[var].unique()[i]) == True:
                next
            else:
                var_unique_list.append(dataframe[var].unique()[i])
                dataframe[var].unique()[i] = dataframe[dataframe[var]==\
                        (dataframe[var].unique()[i])]['saleprice']
        for cat in var_unique_list:
            if pd.isnull(cat) == True:
                next
            else:
                resultdict[var][str(cat)] = \
                dataframe['saleprice'][dataframe[var]\
                                       == cat].mean()
        f, p = stats.f_oneway(*(dataframe['saleprice'][dataframe[var]\
                                    == cat] for cat in var_unique_list))
        resultdict[var]['statistic'] = f
        resultdict[var]['P'] = p 
    return resultdict

In [179]:
ANOVA_list(cat_list, df, cat_result_dict)

{'street': {'Pave': 181793.56555772995,
  'Grvl': 86901.42857142857,
  'statistic': 3.169173295799292,
  'P': 0.0015512882730560206,
  'significance': 'Y'},
 'centralair': {'Y': 187441.50366492147,
  'N': 100575.0780141844,
  'statistic': 170.78696230993063,
  'P': 1.5113094019793903e-37},
 'mszoning': {'RL': 191235.16458072592,
  'RM': 129119.14240506329,
  'FV': 218618.31683168316,
  'C (all)': 78010.8947368421,
  'A (agr)': 47300.0,
  'RH': 148859.57142857142,
  'I (all)': 57625.0,
  'statistic': 42.46657889196624,
  'P': 4.706623428740341e-49},
 'lotshape': {'IR1': 211848.67052023122,
  'Reg': 162925.81235521234,
  'IR2': 233954.74545454545,
  'IR3': 193182.22222222222,
  'statistic': 72.48679839510044,
  'P': 1.491278709891155e-44},
 'landcontour': {'Lvl': 178998.5648399349,
  'HLS': 262353.37647058826,
  'Bnk': 142868.775,
  'Low': 199313.11627906977,
  'statistic': 39.23484353043047,
  'P': 1.1956767644954456e-24},
 'utilities': {'AllPub': 181551.60224499757,
  'NoSeWa': 137500.

In [180]:
# Iterate the dictionary to provide the list of significant values in the category list

significantlist = []
for key1, dict1 in cat_result_dict.items(): 
    for key2, value1 in dict1.items():
        if key2 == 'P':
            if value1 < 0.05:
                significantlist.append(key1)
        else: 
            next
significantlist

['street',
 'centralair',
 'mszoning',
 'lotshape',
 'landcontour',
 'lotconfig',
 'landslope',
 'neighborhood',
 'condition1',
 'condition2',
 'bldgtype',
 'housestyle',
 'roofstyle',
 'roofmatl',
 'exterior1st',
 'exterior2nd',
 'masvnrtype',
 'exterqual',
 'extercond',
 'foundation',
 'bsmtqual',
 'bsmtcond',
 'bsmtexposure',
 'bsmtfintype1',
 'bsmtfintype2',
 'heating',
 'heatingqc',
 'electrical',
 'kitchenqual',
 'functional',
 'garagetype',
 'garagefinish',
 'garagequal',
 'garagecond',
 'paveddrive',
 'saletype']

In [181]:
# Create a dictionary of variables with highest statistics

top_Fstatistic_dict = {}
index = []
count = 0
for key1, dict1 in cat_result_dict.items(): 
    if dict1['P'] <= 0.05:
        top_Fstatistic_dict[key1] = dict1['statistic']
        index.append(count)
        count += 1
    else: 
        next
Fstat_df = pd.DataFrame(list(top_Fstatistic_dict.items()), index = index, columns = ["variable", "stat"])

In [182]:
# Creating a list of categorical variables with high T or F statistic

top_cat_list = list(Fstat_df.sort_values(by = 'stat', ascending = False).nlargest(15,['stat'])['variable'])

In [183]:
# Remove related variables 

top_cat_list.remove('bsmtexposure') # variable was removed due to similarity to other variables
top_cat_list.remove('bsmtfintype1') # variable was removed due to similarity to other variables
top_cat_list.remove('garagetype') # variable was removed due to similarity to other variables

top_cat_list

['exterqual',
 'kitchenqual',
 'bsmtqual',
 'garagefinish',
 'masvnrtype',
 'centralair',
 'foundation',
 'heatingqc',
 'neighborhood',
 'paveddrive',
 'lotshape',
 'saletype']

In [184]:
df[top_cat_list].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2051 entries, 109 to 10
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   exterqual     2051 non-null   object
 1   kitchenqual   2051 non-null   object
 2   bsmtqual      1996 non-null   object
 3   garagefinish  1937 non-null   object
 4   masvnrtype    2029 non-null   object
 5   centralair    2051 non-null   object
 6   foundation    2051 non-null   object
 7   heatingqc     2051 non-null   object
 8   neighborhood  2051 non-null   object
 9   paveddrive    2051 non-null   object
 10  lotshape      2051 non-null   object
 11  saletype      2051 non-null   object
dtypes: object(12)
memory usage: 208.3+ KB


In [185]:
# Fill in the missing variables

print(df.bsmtqual.mode())
print(df.garagefinish.mode())
print(df.masvnrtype.mode())

0    TA
dtype: object
0    Unf
dtype: object
0    None
dtype: object


#### To impute the missing entries for categorical variables, the mode of each column will be used.



In [187]:
# Replacing all NaN with mode

df['bsmtqual'].fillna('TA', inplace = True)
df['garagefinish'].fillna('Unf', inplace = True)
df['masvnrtype'].fillna('None', inplace = True)

In [189]:
df[top_cat_list].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2051 entries, 109 to 10
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   exterqual     2051 non-null   object
 1   kitchenqual   2051 non-null   object
 2   bsmtqual      2051 non-null   object
 3   garagefinish  2051 non-null   object
 4   masvnrtype    2051 non-null   object
 5   centralair    2051 non-null   object
 6   foundation    2051 non-null   object
 7   heatingqc     2051 non-null   object
 8   neighborhood  2051 non-null   object
 9   paveddrive    2051 non-null   object
 10  lotshape      2051 non-null   object
 11  saletype      2051 non-null   object
dtypes: object(12)
memory usage: 208.3+ KB


### Imputing missing values in continuous variables

Missing entries in continuous variables are imputed with the median or mean of the variable depending if it is normally distributed.

In [190]:
# finding which categorical variable has nan unique cat

na_list = []
for col in top_corr_list:
    for cat in df[col].unique():
        if type(cat) == float:
            na_list.append(col)
print('categories with nan entries', na_list)

categories with nan entries []


In [191]:
df[final_corr_list].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2051 entries, 109 to 10
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   overallqual   2051 non-null   int64  
 1   masvnrarea    2029 non-null   float64
 2   poolarea      2051 non-null   int64  
 3   halfbath      2051 non-null   int64  
 4   fireplaces    2051 non-null   int64  
 5   openporchsf   2051 non-null   int64  
 6   grlivarea     2051 non-null   int64  
 7   screenporch   2051 non-null   int64  
 8   lotarea       2051 non-null   int64  
 9   3ssnporch     2051 non-null   int64  
 10  bsmtfullbath  2049 non-null   float64
 11  totalbsmtsf   2050 non-null   float64
 12  garagearea    2050 non-null   float64
 13  yearbuilt     2051 non-null   int64  
dtypes: float64(4), int64(10)
memory usage: 240.4 KB


In [193]:
df.masvnrarea.hist(figsize = (5,5), bins = 20)
print(stats.shapiro(df.masvnrarea.dropna()))

print(df.masvnrarea.median())
df.masvnrarea.fillna(df.masvnrarea.median(), inplace=True)

ShapiroResult(statistic=0.6350643634796143, pvalue=0.0)
0.0


In [194]:
df.bsmtfullbath.hist(figsize = (5,5), bins = 20)
print(stats.shapiro(df.bsmtfullbath.dropna()))

print(df.bsmtfullbath.median())
df.bsmtfullbath.fillna(df.bsmtfullbath.median(), inplace = True)

ShapiroResult(statistic=0.6606760025024414, pvalue=0.0)
0.0


In [195]:
df.totalbsmtsf.hist(figsize = (5,5), bins = 20)
print(stats.shapiro(df.totalbsmtsf.dropna()))

print(df.totalbsmtsf.median())
df.totalbsmtsf.fillna(df.totalbsmtsf.median(), inplace=True)

ShapiroResult(statistic=0.9275934100151062, pvalue=2.227390454751659e-30)
994.5


In [196]:
df.garagearea.hist(figsize = (5,5), bins = 20)
print(stats.shapiro(df.garagearea.dropna())) 

print(df.garagearea.median())
df.garagearea.fillna(df.garagearea.median(), inplace=True)

ShapiroResult(statistic=0.9763252139091492, pvalue=6.247190586439361e-18)
480.0


In [197]:
df[final_corr_list].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2051 entries, 109 to 10
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   overallqual   2051 non-null   int64  
 1   masvnrarea    2051 non-null   float64
 2   poolarea      2051 non-null   int64  
 3   halfbath      2051 non-null   int64  
 4   fireplaces    2051 non-null   int64  
 5   openporchsf   2051 non-null   int64  
 6   grlivarea     2051 non-null   int64  
 7   screenporch   2051 non-null   int64  
 8   lotarea       2051 non-null   int64  
 9   3ssnporch     2051 non-null   int64  
 10  bsmtfullbath  2051 non-null   float64
 11  totalbsmtsf   2051 non-null   float64
 12  garagearea    2051 non-null   float64
 13  yearbuilt     2051 non-null   int64  
dtypes: float64(4), int64(10)
memory usage: 240.4 KB


### Changing the variables into dummy coded variables. For the ease of doing linear regression later. 

In [198]:
interest= top_cat_list+final_corr_list
interest.append('saleprice')

In [199]:
interest.remove('centralair')
interest.remove('openporchsf')
interest.remove('halfbath')
interest.remove('foundation')
interest.remove('saletype')

print(len(interest))
interestdf = df[interest]

22


In [200]:
interestdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2051 entries, 109 to 10
Data columns (total 22 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   exterqual     2051 non-null   object 
 1   kitchenqual   2051 non-null   object 
 2   bsmtqual      2051 non-null   object 
 3   garagefinish  2051 non-null   object 
 4   masvnrtype    2051 non-null   object 
 5   heatingqc     2051 non-null   object 
 6   neighborhood  2051 non-null   object 
 7   paveddrive    2051 non-null   object 
 8   lotshape      2051 non-null   object 
 9   overallqual   2051 non-null   int64  
 10  masvnrarea    2051 non-null   float64
 11  poolarea      2051 non-null   int64  
 12  fireplaces    2051 non-null   int64  
 13  grlivarea     2051 non-null   int64  
 14  screenporch   2051 non-null   int64  
 15  lotarea       2051 non-null   int64  
 16  3ssnporch     2051 non-null   int64  
 17  bsmtfullbath  2051 non-null   float64
 18  totalbsmtsf   2051 non-null 

In [201]:
# Creating an ordinal dictionary to determine the unicode value 

qual = {'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1, 'NA':0}

In [202]:
# Mapping qual to all qual variables

interestdf['exterq'] = interestdf.exterqual.map(qual)
interestdf = interestdf.drop('exterqual', axis = 1)
interestdf['heatingq'] = interestdf.heatingqc.map(qual)
interestdf = interestdf.drop('heatingqc', axis = 1)
interestdf['bsmtq'] = interestdf.bsmtqual.map(qual)
interestdf = interestdf.drop('bsmtqual', axis = 1)
interestdf['kitchenq'] = interestdf.kitchenqual.map(qual)
interestdf = interestdf.drop('kitchenqual', axis = 1)

<ipython-input-202-9158ed21b9c2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  interestdf['exterq'] = interestdf.exterqual.map(qual)


In [203]:
interest = list(interestdf.columns)
interest

['garagefinish',
 'masvnrtype',
 'neighborhood',
 'paveddrive',
 'lotshape',
 'overallqual',
 'masvnrarea',
 'poolarea',
 'fireplaces',
 'grlivarea',
 'screenporch',
 'lotarea',
 '3ssnporch',
 'bsmtfullbath',
 'totalbsmtsf',
 'garagearea',
 'yearbuilt',
 'saleprice',
 'exterq',
 'heatingq',
 'bsmtq',
 'kitchenq']

In [204]:
interestdf.head()

,garagefinish,masvnrtype,neighborhood,paveddrive,lotshape,overallqual,masvnrarea,poolarea,fireplaces,grlivarea,...,3ssnporch,bsmtfullbath,totalbsmtsf,garagearea,yearbuilt,saleprice,exterq,heatingq,bsmtq,kitchenq
Id,,,,,,,,,,,,,,,,,,,,,
109,RFn,BrkFace,Sawyer,Y,IR1,6,289.0,0,0,1479,...,0,0.0,725.0,475.0,1976,130500,4,5,3,4
544,RFn,BrkFace,SawyerW,Y,IR1,7,132.0,0,1,2122,...,0,1.0,913.0,559.0,1996,220000,4,5,4,4
153,Unf,None,NAmes,Y,Reg,5,0.0,0,0,1057,...,0,1.0,1057.0,246.0,1953,109000,3,3,3,4
318,Fin,None,Timber,Y,Reg,5,0.0,0,0,1444,...,0,0.0,384.0,400.0,2006,174000,3,4,4,3
255,Unf,None,SawyerW,N,IR1,6,0.0,0,0,1445,...,0,0.0,676.0,484.0,1900,138500,3,3,2,3


In [205]:
interestdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2051 entries, 109 to 10
Data columns (total 22 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   garagefinish  2051 non-null   object 
 1   masvnrtype    2051 non-null   object 
 2   neighborhood  2051 non-null   object 
 3   paveddrive    2051 non-null   object 
 4   lotshape      2051 non-null   object 
 5   overallqual   2051 non-null   int64  
 6   masvnrarea    2051 non-null   float64
 7   poolarea      2051 non-null   int64  
 8   fireplaces    2051 non-null   int64  
 9   grlivarea     2051 non-null   int64  
 10  screenporch   2051 non-null   int64  
 11  lotarea       2051 non-null   int64  
 12  3ssnporch     2051 non-null   int64  
 13  bsmtfullbath  2051 non-null   float64
 14  totalbsmtsf   2051 non-null   float64
 15  garagearea    2051 non-null   float64
 16  yearbuilt     2051 non-null   int64  
 17  saleprice     2051 non-null   int64  
 18  exterq        2051 non-null 

In [206]:
interest.remove('saleprice') 
interest.remove('neighborhood') 
interestdf = interestdf.drop('neighborhood', axis = 1)

In [207]:
# Making dummy variables

interestdf = pd.get_dummies(interestdf, drop_first = True)

In [208]:
interest_list = [ col for col in interestdf.columns if col !='saleprice']

In [209]:
interest_list

['overallqual',
 'masvnrarea',
 'poolarea',
 'fireplaces',
 'grlivarea',
 'screenporch',
 'lotarea',
 '3ssnporch',
 'bsmtfullbath',
 'totalbsmtsf',
 'garagearea',
 'yearbuilt',
 'exterq',
 'heatingq',
 'bsmtq',
 'kitchenq',
 'garagefinish_RFn',
 'garagefinish_Unf',
 'masvnrtype_BrkFace',
 'masvnrtype_None',
 'masvnrtype_Stone',
 'paveddrive_P',
 'paveddrive_Y',
 'lotshape_IR2',
 'lotshape_IR3',
 'lotshape_Reg']

## Linear Regression Modelling 

In [210]:
# Linear Regression class 

lr = LinearRegression()

In [211]:
X = interestdf[interest_list]
y = interestdf['saleprice']

In [212]:
# Splitting current training dataset to do smaller training datasets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [213]:
X_test.shape

(616, 26)

In [214]:
# Scaling the variables to an equal scale

ss = StandardScaler()
ss.fit(X_train)
X_train_sc=ss.transform(X_train)
X_test_sc = ss.transform(X_test)

In [215]:
model = lr.fit(X_train_sc,y_train)
score = model.score(X_train_sc,y_train)

predictions = model.predict(X_train_sc)

In [216]:
# Plotting the fit of the predicted vs the actual sale price

plt.figure(figsize = (15,15))
ax = sns.regplot(predictions, y_train)
ax.plot()
ax.set_title('Scatterplot of Predicted Sales Price vs Actual Sales Price', size = 20)
ax.set_ylabel('Actual Sale Price', size = 16)
ax.set_xlabel('Predicted Sale Price', size = 16)
plt.yticks(size = 15)
plt.xticks(size = 15)

C:\Users\singh\anaconda3\lib\site-packages\seaborn\_decorators.py:36: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  warnings.warn(


(array([-100000.,       0.,  100000.,  200000.,  300000.,  400000.,
         500000.,  600000.,  700000.]),
 [Text(0, 0, ''),
  Text(0, 0, ''),
  Text(0, 0, ''),
  Text(0, 0, ''),
  Text(0, 0, ''),
  Text(0, 0, ''),
  Text(0, 0, ''),
  Text(0, 0, ''),
  Text(0, 0, '')])

In [217]:
print(model.coef_)
print(model.intercept_)
np.sqrt(mean_squared_error(y_train, predictions))

[18447.39036742  7626.78706546 -4611.06981767  6167.51327616
 18287.92271429  5302.32361032  6414.10376294  1258.46646932
  6388.6997598   4708.51531307  6353.50048852  2249.08074479
  8528.79413695  2963.57873536  6876.10107685  8623.92468998
 -3526.94036972 -2920.60768114  4508.61901375  8744.53095479
  6277.23689358  -371.14684497   857.60846648   450.06243836
 -5894.71965442 -1294.05209613]
180717.9693379791


32933.693008226874

In [218]:
n = 2051
p = len(interest_list)

In [219]:
# Adj r2 value for training set

adjr2_lr1 = 1-(1-(score))*(n-1)/(n-p-1)
adjr2_lr1

0.8285210802803271

In [220]:
score2 =lr.score(X_test_sc, y_test)

prediction2 = model.predict(X_test_sc)

# Adj r2 value for test set

adjr2_lr2 = 1-(1-(score2))*(n-1)/(n-p-1)
adjr2_lr2 

0.8587401232575231

In [221]:
# RMSE value
np.sqrt(mean_squared_error(y_test, prediction2)) 

28873.197062950938

In [222]:
# Plotting the fit of the predicted vs the actual sale price

plt.figure(figsize = (10,10))
ax = sns.regplot(prediction2, y_test)
ax.plot()
ax.set_title('Scatterplot of Predicted Sales Price vs Actual Sales Price', size = 20)
ax.set_ylabel('Actual Sale Price', size = 16)
ax.set_xlabel('Predicted Sale Price', size = 16)
plt.yticks(size = 15)
plt.xticks(size = 15)

C:\Users\singh\anaconda3\lib\site-packages\seaborn\_decorators.py:36: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  warnings.warn(


(array([     0.,  50000., 100000., 150000., 200000., 250000., 300000.,
        350000., 400000., 450000.]),
 [Text(0, 0, ''),
  Text(0, 0, ''),
  Text(0, 0, ''),
  Text(0, 0, ''),
  Text(0, 0, ''),
  Text(0, 0, ''),
  Text(0, 0, ''),
  Text(0, 0, ''),
  Text(0, 0, ''),
  Text(0, 0, '')])

In [223]:
print(model.coef_)
print(model.intercept_)

[18447.39036742  7626.78706546 -4611.06981767  6167.51327616
 18287.92271429  5302.32361032  6414.10376294  1258.46646932
  6388.6997598   4708.51531307  6353.50048852  2249.08074479
  8528.79413695  2963.57873536  6876.10107685  8623.92468998
 -3526.94036972 -2920.60768114  4508.61901375  8744.53095479
  6277.23689358  -371.14684497   857.60846648   450.06243836
 -5894.71965442 -1294.05209613]
180717.9693379791


In [224]:
# Cross validation scores on train dataset
lr_scores = cross_val_score(lr, X_train_sc, y_train, cv = 5)
print(lr_scores)
lr_scores.mean()

[0.83951236 0.81776343 0.86044835 0.78956273 0.65378331]


0.7922140359510348

In [225]:
# Cross validation scores on test dataset
lr2_scores = cross_val_score(lr, X_test_sc, y_test, cv = 5)
print(lr2_scores)
lr2_scores.mean()

[0.86567014 0.89481436 0.85250138 0.8667611  0.87665334]


0.8712800640668593

### Lasso Regression 

In [226]:
# Initialising Lasso Regression in Scikit

ls = LassoCV(n_alphas = 200)

In [227]:
X_train_sc.shape

(1435, 26)

In [228]:
# Cross validating Lasso Regression with training set.

ls_scores = cross_val_score(ls, X_train_sc, y_train, cv = 5)
print(ls_scores)
ls_scores.mean()

[0.83955727 0.81752211 0.86037509 0.79006151 0.64848657]


0.7912005099538054

In [229]:
X_train_sc

array([[-6.79941799e-02,  1.85542776e+00, -6.83356918e-02, ...,
        -1.55783118e-01, -7.00140042e-02,  7.60695646e-01],
       [ 6.28946165e-01, -5.56076337e-01, -6.83356918e-02, ...,
        -1.55783118e-01, -7.00140042e-02, -1.31458620e+00],
       [-7.64934524e-01, -5.56076337e-01, -6.83356918e-02, ...,
        -1.55783118e-01, -7.00140042e-02,  7.60695646e-01],
       ...,
       [-6.79941799e-02,  5.66226501e-03,  1.60106099e+01, ...,
        -1.55783118e-01, -7.00140042e-02,  7.60695646e-01],
       [-6.79941799e-02,  1.64537829e-01, -6.83356918e-02, ...,
        -1.55783118e-01, -7.00140042e-02,  7.60695646e-01],
       [ 6.28946165e-01,  8.28410723e-01, -6.83356918e-02, ...,
        -1.55783118e-01, -7.00140042e-02,  7.60695646e-01]])

In [230]:
# Fit the lasson model

ls.fit(X_train_sc, y_train)

LassoCV(n_alphas=200)

In [231]:
ls.score(X_train_sc, y_train)

0.8305734030472174

In [232]:
# Cross validating Lasso Regression with test data

ls_scores2 = cross_val_score(ls, X_test_sc, y_test, cv = 5)
print(ls_scores2)
ls_scores2.mean()

[0.87349978 0.89448194 0.84976102 0.8629153  0.8738559 ]


0.8709027871149502

In [233]:
score2 = ls.score(X_test_sc, y_test)

In [234]:
predls = ls.predict(X_test_sc)

In [235]:
ls_test_r2score = r2_score(y_test, predls)
ls_test_r2score

0.8603003007868635

In [236]:
# RMSE
np.sqrt(mean_squared_error(y_test, predls))

28897.140894843284

In [237]:
sum(ls.coef_ != 0)

25

In [238]:
# Finding for ajusted 

p = np.sum(ls.coef_ != 0)
adjr2_ls2= 1-(1-(ls_test_r2score))*(n-1)/(n-p-1)
adjr2_ls2 

0.8585756131422568

In [239]:
pd.Series(ls.coef_, index = interest_list).sort_values(ascending =True).plot.bar(figsize=(15,7))

<AxesSubplot:title={'center':'Scatterplot of Predicted Sales Price vs Actual Sales Price'}, xlabel='Predicted Sale Price', ylabel='Actual Sale Price'>

In [240]:
# Plotting out the residuals against the predicted y

residual_ls = y_test - predls

In [241]:
plt.scatter(predls, residual_ls)
plt.title('Lasso Regression chart of residuals vs predicted y', size = 20)
plt.yticks( size = 15)
plt.xticks(size = 15)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25]),
 [Text(0, 0, 'lotshape_IR3'),
  Text(1, 0, 'poolarea'),
  Text(2, 0, 'garagefinish_RFn'),
  Text(3, 0, 'garagefinish_Unf'),
  Text(4, 0, 'lotshape_Reg'),
  Text(5, 0, 'paveddrive_P'),
  Text(6, 0, 'masvnrtype_BrkFace'),
  Text(7, 0, 'lotshape_IR2'),
  Text(8, 0, 'paveddrive_Y'),
  Text(9, 0, '3ssnporch'),
  Text(10, 0, 'yearbuilt'),
  Text(11, 0, 'heatingq'),
  Text(12, 0, 'masvnrtype_Stone'),
  Text(13, 0, 'masvnrtype_None'),
  Text(14, 0, 'totalbsmtsf'),
  Text(15, 0, 'screenporch'),
  Text(16, 0, 'fireplaces'),
  Text(17, 0, 'lotarea'),
  Text(18, 0, 'bsmtfullbath'),
  Text(19, 0, 'garagearea'),
  Text(20, 0, 'bsmtq'),
  Text(21, 0, 'masvnrarea'),
  Text(22, 0, 'exterq'),
  Text(23, 0, 'kitchenq'),
  Text(24, 0, 'grlivarea'),
  Text(25, 0, 'overallqual')])

### Ridge Regression

In [242]:
# Ridge Regression 

rd = RidgeCV(alphas = np.linspace(0.1, 10, 100))

In [243]:
rd_scores = cross_val_score(rd, X_train_sc, y_train, cv = 3)
print(rd_scores)
rd_scores.mean()

[0.83331456 0.83849588 0.65675411]


0.7761881818800488

In [244]:
print(rd.fit(X_train_sc, y_train))
print(rd.score(X_train_sc, y_train))

RidgeCV(alphas=array([ 0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9,  1. ,  1.1,
        1.2,  1.3,  1.4,  1.5,  1.6,  1.7,  1.8,  1.9,  2. ,  2.1,  2.2,
        2.3,  2.4,  2.5,  2.6,  2.7,  2.8,  2.9,  3. ,  3.1,  3.2,  3.3,
        3.4,  3.5,  3.6,  3.7,  3.8,  3.9,  4. ,  4.1,  4.2,  4.3,  4.4,
        4.5,  4.6,  4.7,  4.8,  4.9,  5. ,  5.1,  5.2,  5.3,  5.4,  5.5,
        5.6,  5.7,  5.8,  5.9,  6. ,  6.1,  6.2,  6.3,  6.4,  6.5,  6.6,
        6.7,  6.8,  6.9,  7. ,  7.1,  7.2,  7.3,  7.4,  7.5,  7.6,  7.7,
        7.8,  7.9,  8. ,  8.1,  8.2,  8.3,  8.4,  8.5,  8.6,  8.7,  8.8,
        8.9,  9. ,  9.1,  9.2,  9.3,  9.4,  9.5,  9.6,  9.7,  9.8,  9.9,
       10. ]))
0.8306598383643353


In [245]:
rd_scores2 = cross_val_score(rd, X_test_sc, y_test, cv = 3)
print(rd_scores2)
rd_scores2.mean()

[0.87084436 0.85864779 0.8749455 ]


0.8681458827403397

In [246]:
rd.score(X_train_sc, y_train)

0.8306598383643353

In [247]:
predrd = rd.predict(X_test_sc)

In [248]:
rdr2score= r2_score(y_test, predrd)
rdr2score

0.8603505477730229

In [249]:
# RMSE 

np.sqrt(mean_squared_error(y_test, predrd)) 

28891.943586498834

In [250]:
pd.Series(rd.coef_, index = interest_list).sort_values(ascending = True).plot.bar(figsize=(15,7))

<AxesSubplot:title={'center':'Lasso Regression chart of residuals vs predicted y'}, xlabel='Predicted Sale Price', ylabel='Actual Sale Price'>

In [251]:
p = np.sum(rd.coef_ != 0)
adjr2_rd = 1-(1-(rdr2score))*(n-1)/(n-p-1)
adjr2_rd

0.8585566318847316

In [252]:
# Plotting out the residuals against the predicted y

residuals = y_test - predrd

In [253]:
plt.scatter(predrd, residuals)
plt.title('Ridge Regression chart of residuals vs predicted y', size = 20)
plt.yticks( size = 15)
plt.xticks(size = 15)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25]),
 [Text(0, 0, 'lotshape_IR3'),
  Text(1, 0, 'poolarea'),
  Text(2, 0, 'garagefinish_RFn'),
  Text(3, 0, 'garagefinish_Unf'),
  Text(4, 0, 'lotshape_Reg'),
  Text(5, 0, 'paveddrive_P'),
  Text(6, 0, 'lotshape_IR2'),
  Text(7, 0, 'paveddrive_Y'),
  Text(8, 0, '3ssnporch'),
  Text(9, 0, 'yearbuilt'),
  Text(10, 0, 'masvnrtype_BrkFace'),
  Text(11, 0, 'heatingq'),
  Text(12, 0, 'totalbsmtsf'),
  Text(13, 0, 'masvnrtype_Stone'),
  Text(14, 0, 'screenporch'),
  Text(15, 0, 'fireplaces'),
  Text(16, 0, 'masvnrtype_None'),
  Text(17, 0, 'bsmtfullbath'),
  Text(18, 0, 'lotarea'),
  Text(19, 0, 'garagearea'),
  Text(20, 0, 'bsmtq'),
  Text(21, 0, 'masvnrarea'),
  Text(22, 0, 'exterq'),
  Text(23, 0, 'kitchenq'),
  Text(24, 0, 'grlivarea'),
  Text(25, 0, 'overallqual')])

### Test Data

Test data was cleaned and explored
Lasso regression was to be done as it had produced the best r2 value and RMSE for the split test data

In [257]:
test = pd.read_csv('../datasets/test.csv', index_col='Id')

In [258]:
# Data structure for test data
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 878 entries, 2658 to 1939
Data columns (total 79 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   PID              878 non-null    int64  
 1   MS SubClass      878 non-null    int64  
 2   MS Zoning        878 non-null    object 
 3   Lot Frontage     718 non-null    float64
 4   Lot Area         878 non-null    int64  
 5   Street           878 non-null    object 
 6   Alley            58 non-null     object 
 7   Lot Shape        878 non-null    object 
 8   Land Contour     878 non-null    object 
 9   Utilities        878 non-null    object 
 10  Lot Config       878 non-null    object 
 11  Land Slope       878 non-null    object 
 12  Neighborhood     878 non-null    object 
 13  Condition 1      878 non-null    object 
 14  Condition 2      878 non-null    object 
 15  Bldg Type        878 non-null    object 
 16  House Style      878 non-null    object 
 17  Overall Qual

In [259]:
test.shape

(878, 79)

### Drop data that are significantly incomplete by >60%

In [260]:
test = test.drop('Alley', axis = 1)

In [261]:
test = test.drop('Pool QC', axis = 1)

In [262]:
test = test.drop("Fence", axis = 1)

In [263]:
test = test.drop("Misc Feature", axis = 1)

In [264]:
test = test.drop("Fireplace Qu", axis = 1)

In [265]:
test = test.drop("PID", axis = 1)

In [266]:
test.head()

,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,...,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,Sale Type
Id,,,,,,,,,,,,,,,,,,,,,
2658,190,RM,69.0,9142,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,60,112,0,0,0,0,4,2006,WD
2718,90,RL,NaN,9662,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,170,0,0,0,0,0,0,8,2006,WD
2414,60,RL,58.0,17104,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,100,24,0,0,0,0,0,9,2006,New
1989,30,RM,60.0,8520,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,184,0,0,0,0,7,2007,WD
625,20,RL,NaN,9500,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,76,0,0,185,0,0,7,2009,WD


### Renaming test data columns 

In [267]:
test_col = test.columns
test_rename_dict = {}
for col in test_col: 
    test_rename_dict[col] = col.lower().replace(' ', "")

test.rename(columns = test_rename_dict, inplace = True)
test.columns

Index(['mssubclass', 'mszoning', 'lotfrontage', 'lotarea', 'street',
       'lotshape', 'landcontour', 'utilities', 'lotconfig', 'landslope',
       'neighborhood', 'condition1', 'condition2', 'bldgtype', 'housestyle',
       'overallqual', 'overallcond', 'yearbuilt', 'yearremod/add', 'roofstyle',
       'roofmatl', 'exterior1st', 'exterior2nd', 'masvnrtype', 'masvnrarea',
       'exterqual', 'extercond', 'foundation', 'bsmtqual', 'bsmtcond',
       'bsmtexposure', 'bsmtfintype1', 'bsmtfinsf1', 'bsmtfintype2',
       'bsmtfinsf2', 'bsmtunfsf', 'totalbsmtsf', 'heating', 'heatingqc',
       'centralair', 'electrical', '1stflrsf', '2ndflrsf', 'lowqualfinsf',
       'grlivarea', 'bsmtfullbath', 'bsmthalfbath', 'fullbath', 'halfbath',
       'bedroomabvgr', 'kitchenabvgr', 'kitchenqual', 'totrmsabvgrd',
       'functional', 'fireplaces', 'garagetype', 'garageyrblt', 'garagefinish',
       'garagecars', 'garagearea', 'garagequal', 'garagecond', 'paveddrive',
       'wooddecksf', 'openporchsf

In [268]:
# Mapping qual variables to qual testing

test['exterq'] = test.exterqual.map(qual)
test = test.drop('exterqual', axis = 1)
test['heatingq'] = test.heatingqc.map(qual)
test = test.drop('heatingqc', axis = 1)
test['bsmtq'] = test.bsmtqual.map(qual)
test = test.drop('bsmtqual', axis = 1)
test['kitchenq'] = test.kitchenqual.map(qual)
test = test.drop('kitchenqual', axis = 1)

In [269]:
interesttest = [x for x in interest if x != 'saleprice']
len(interesttest)

20

In [270]:
len(interesttest)

20

In [271]:
# Create df for test csv
test_interest_only = test[interesttest]

In [272]:
# Structure the dataset
test_interest_only.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 878 entries, 2658 to 1939
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   garagefinish  833 non-null    object 
 1   masvnrtype    877 non-null    object 
 2   paveddrive    878 non-null    object 
 3   lotshape      878 non-null    object 
 4   overallqual   878 non-null    int64  
 5   masvnrarea    877 non-null    float64
 6   poolarea      878 non-null    int64  
 7   fireplaces    878 non-null    int64  
 8   grlivarea     878 non-null    int64  
 9   screenporch   878 non-null    int64  
 10  lotarea       878 non-null    int64  
 11  3ssnporch     878 non-null    int64  
 12  bsmtfullbath  878 non-null    int64  
 13  totalbsmtsf   878 non-null    int64  
 14  garagearea    878 non-null    int64  
 15  yearbuilt     878 non-null    int64  
 16  exterq        878 non-null    int64  
 17  heatingq      878 non-null    int64  
 18  bsmtq         853 non-null

In [273]:
# Filling in the missing variables

print(test_interest_only.garagefinish.mode())
print(test_interest_only.masvnrtype.mode())

0    Unf
dtype: object
0    None
dtype: object


In [274]:
# Replacing all NaN with mode

test_interest_only['garagefinish'].fillna('Unf', inplace = True)
test_interest_only['masvnrtype'].fillna('None', inplace = True)

C:\Users\singh\anaconda3\lib\site-packages\pandas\core\series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [275]:
test_interest_only.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 878 entries, 2658 to 1939
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   garagefinish  878 non-null    object 
 1   masvnrtype    878 non-null    object 
 2   paveddrive    878 non-null    object 
 3   lotshape      878 non-null    object 
 4   overallqual   878 non-null    int64  
 5   masvnrarea    877 non-null    float64
 6   poolarea      878 non-null    int64  
 7   fireplaces    878 non-null    int64  
 8   grlivarea     878 non-null    int64  
 9   screenporch   878 non-null    int64  
 10  lotarea       878 non-null    int64  
 11  3ssnporch     878 non-null    int64  
 12  bsmtfullbath  878 non-null    int64  
 13  totalbsmtsf   878 non-null    int64  
 14  garagearea    878 non-null    int64  
 15  yearbuilt     878 non-null    int64  
 16  exterq        878 non-null    int64  
 17  heatingq      878 non-null    int64  
 18  bsmtq         853 non-null

In [276]:
# Creating dummy variables for categorical variables
test_interest_only = pd.get_dummies(test_interest_only, drop_first = True)

In [277]:
test_interest_only.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 878 entries, 2658 to 1939
Data columns (total 27 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   overallqual         878 non-null    int64  
 1   masvnrarea          877 non-null    float64
 2   poolarea            878 non-null    int64  
 3   fireplaces          878 non-null    int64  
 4   grlivarea           878 non-null    int64  
 5   screenporch         878 non-null    int64  
 6   lotarea             878 non-null    int64  
 7   3ssnporch           878 non-null    int64  
 8   bsmtfullbath        878 non-null    int64  
 9   totalbsmtsf         878 non-null    int64  
 10  garagearea          878 non-null    int64  
 11  yearbuilt           878 non-null    int64  
 12  exterq              878 non-null    int64  
 13  heatingq            878 non-null    int64  
 14  bsmtq               853 non-null    float64
 15  kitchenq            878 non-null    int64  
 16  gara

In [278]:
test_interest_only.masvnrarea.hist(bins=20, figsize = (7,7))
print(test_interest_only.masvnrarea.median())
test_interest_only['masvnrarea'].fillna(test_interest_only.masvnrarea.median(), inplace = True)

0.0


In [279]:
test_interest_only.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 878 entries, 2658 to 1939
Data columns (total 27 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   overallqual         878 non-null    int64  
 1   masvnrarea          878 non-null    float64
 2   poolarea            878 non-null    int64  
 3   fireplaces          878 non-null    int64  
 4   grlivarea           878 non-null    int64  
 5   screenporch         878 non-null    int64  
 6   lotarea             878 non-null    int64  
 7   3ssnporch           878 non-null    int64  
 8   bsmtfullbath        878 non-null    int64  
 9   totalbsmtsf         878 non-null    int64  
 10  garagearea          878 non-null    int64  
 11  yearbuilt           878 non-null    int64  
 12  exterq              878 non-null    int64  
 13  heatingq            878 non-null    int64  
 14  bsmtq               853 non-null    float64
 15  kitchenq            878 non-null    int64  
 16  gara

In [280]:
test_interest_only.bsmtq.hist(bins=20, figsize = (7,7))
print(stats.shapiro(test_interest_only.bsmtq.dropna()))

print(test_interest_only.bsmtq.median())
test_interest_only['bsmtq'].fillna(test_interest_only.bsmtq.median(), inplace = True)

ShapiroResult(statistic=0.820310115814209, pvalue=8.099883567109398e-30)
4.0


In [281]:
test_interest_only = test_interest_only.drop('masvnrtype_CBlock', axis = 1)

In [282]:
# Rename the df

X_test_test = test_interest_only

print(test_interest_only.shape)
print(test_interest_only.info())

(878, 26)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 878 entries, 2658 to 1939
Data columns (total 26 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   overallqual         878 non-null    int64  
 1   masvnrarea          878 non-null    float64
 2   poolarea            878 non-null    int64  
 3   fireplaces          878 non-null    int64  
 4   grlivarea           878 non-null    int64  
 5   screenporch         878 non-null    int64  
 6   lotarea             878 non-null    int64  
 7   3ssnporch           878 non-null    int64  
 8   bsmtfullbath        878 non-null    int64  
 9   totalbsmtsf         878 non-null    int64  
 10  garagearea          878 non-null    int64  
 11  yearbuilt           878 non-null    int64  
 12  exterq              878 non-null    int64  
 13  heatingq            878 non-null    int64  
 14  bsmtq               878 non-null    float64
 15  kitchenq            878 non-null    int64  

In [283]:
# Scaling the variables to same scale

ss = StandardScaler()
ss.fit(X_test_test)
X_test_test_sc = ss.transform(X_test_test)

X_test_test_sc.shape

(878, 26)

In [284]:
# Predict the Saleprice 
predtestls = ls.predict(X_test_test_sc)
predtestls

array([132367.2605764 , 178685.21262851, 224146.22440483, 117852.56647314,
       207976.37026988,  84999.87051105,  92602.35254233, 143036.48844852,
       205202.25959088, 155630.3782549 , 174418.34748393, 119976.58135032,
       156676.02455986, 282816.4207494 , 155273.02220628, 131672.61801828,
       166181.2433412 , 121208.68549364, 192906.93712419, 210235.31527285,
       149776.85798926, 131639.95523091, 219701.66476004, 178009.19522515,
       196552.95265335, 101902.9679044 , 130714.51598859, 133462.85831331,
       152376.550647  ,  40733.95458139,  98091.6532177 ,  98874.06485384,
       243842.06057592, 154112.750672  , 236191.56104789, 198161.03127907,
       100252.34170213,  77913.46286101, 125478.78709803, 207185.02050147,
       179082.48670838, 229709.30764497, 153954.80438582, 183575.15473965,
       249857.7967758 ,  94286.45968369, 237581.075513  , 115966.41895342,
       139119.83921253, 120267.92450871, 109506.31571241, 209980.17256992,
       260027.19279134, 1

In [285]:
# Create a new column for the predicted sale price
test_interest_only['PredSalePrice'] = predtestls

In [286]:
# Exporting out the predicted saleprice values out into a csv
test_interest_only.PredSalePrice.to_csv('../datasets/result.csv', index_label = ['Id'], header = ['PredSalePrice'])

### Conclusion 

In [287]:
interest

['garagefinish',
 'masvnrtype',
 'paveddrive',
 'lotshape',
 'overallqual',
 'masvnrarea',
 'poolarea',
 'fireplaces',
 'grlivarea',
 'screenporch',
 'lotarea',
 '3ssnporch',
 'bsmtfullbath',
 'totalbsmtsf',
 'garagearea',
 'yearbuilt',
 'exterq',
 'heatingq',
 'bsmtq',
 'kitchenq']

In [288]:
conc = pd.DataFrame()
conc['Var'] = interest_list
conc['Coeff'] = ls.coef_
conc.sort_values(by="Coeff").head(50)

,Var,Coeff
24,lotshape_IR3,-5813.508010
2,poolarea,-4530.490761
16,garagefinish_RFn,-3370.029946
17,garagefinish_Unf,-2773.940053
25,lotshape_Reg,-1275.607960
21,paveddrive_P,-352.098436
18,masvnrtype_BrkFace,0.000000
23,lotshape_IR2,414.147965
22,paveddrive_Y,852.998614
7,3ssnporch,1198.223606


### This shows the folowing variables determines the housing prices in Ames

In [289]:
print(interest_list)

['overallqual', 'masvnrarea', 'poolarea', 'fireplaces', 'grlivarea', 'screenporch', 'lotarea', '3ssnporch', 'bsmtfullbath', 'totalbsmtsf', 'garagearea', 'yearbuilt', 'exterq', 'heatingq', 'bsmtq', 'kitchenq', 'garagefinish_RFn', 'garagefinish_Unf', 'masvnrtype_BrkFace', 'masvnrtype_None', 'masvnrtype_Stone', 'paveddrive_P', 'paveddrive_Y', 'lotshape_IR2', 'lotshape_IR3', 'lotshape_Reg']


In [290]:
ls.coef_

array([18480.60833805,  7507.64967104, -4530.49076064,  6104.71311454,
       18286.83939593,  5265.01199331,  6355.51069807,  1198.22360635,
        6358.52133063,  4718.49481707,  6371.74545038,  2156.36239421,
        8556.33225455,  2965.60518296,  6963.6265002 ,  8658.31620837,
       -3370.02994567, -2773.94005304,     0.        ,  3884.59938182,
        3608.83651126,  -352.09843647,   852.99861411,   414.14796501,
       -5813.50800954, -1275.60795993])

### End of Code